    # Build a Simple LLm Application with LCEL - Langchain expression Language -> chain component together

    in this quickstart we'll show you how to build a simple LLM application with Langchain , here it translate english to some other languages. This is relatively simple LLM model single LLM plus some prompting , 

In [ ]:
## Open AI api and Open source model -> llama3,gemma2,mistral ---> Groq , Lpu

import os
from dotenv import load_dotenv
load_dotenv()


groq_api_key = os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq

model = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    api_key=groq_api_key,
)

model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7960ac9fb010>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7960ac9f8b50>, model_name='deepseek-r1-distill-llama-70b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [9]:
from langchain_core.messages import HumanMessage
from langchain_core.messages import SystemMessage

messages = [
    SystemMessage(content = " Translate the following English text to French:"),
    HumanMessage(content="I love programming in Python. It is a versatile language that is great for data science and web development."),
]

result = model.invoke(messages)


In [10]:
from langchain_core.output_parsers import StrOutputParser
parser = StrOutputParser()
result = parser.invoke(result)
print(result)

<think>
Okay, I need to translate this English text into French. Let me read it again: "I love programming in Python. It is a versatile language that is great for data science and web development."

First, I'll break it down sentence by sentence. The first sentence is "I love programming in Python." In French, "I love" is "J'adore." "Programming" becomes "la programmation." "In Python" is "en Python." So putting that together, it should be "J'adore la programmation en Python."

Next sentence: "It is a versatile language that is great for data science and web development." "It is" translates to "C'est." "A versatile language" is "un langage polyvalent." "That is great for" would be "qui est excellent pour." "Data science" is "la science des données," and "web development" is "le développement web." So the second sentence becomes "C'est un langage polyvalent qui est excellent pour la science des données et le développement web."

Now, putting both sentences together, I get: "J'adore la p

In [11]:
## Using LCEL - Chain the components 

chain = model | parser
chain.invoke(messages)

'<think>\nOkay, so I need to translate this English sentence into French. Let me look at the first part: "I love programming in Python." The word "I love" in French is "J\'adore." "Programming" translates to "la programmation." So putting that together, it would be "J\'adore la programmation en Python." That makes sense.\n\nNext part: "It is a versatile language that is great for data science and web development." "It is" is "C\'est" in French. "A versatile language" would be "un langage polyvalent." The phrase "that is great for" translates to "qui est excellent pour." Now, "data science" is "la science des données" and "web development" is "le développement web." Putting that all together: "C\'est un langage polyvalent qui est excellent pour la science des données et le développement web."\n\nLet me double-check the grammar. Using "qui est excellent pour" seems correct because it\'s describing the language\'s suitability. Also, "la science des données" is the accurate term for data s

In [12]:
# Instde of this we can use prompt templates 
from langchain_core.prompts import ChatPromptTemplate

generic_template = "Translate the following into {language} :"

prompt = ChatPromptTemplate.from_messages([
    ("system", generic_template),
    ("user", "{text}"),
])

In [14]:
result = prompt.invoke({
    "language": "French",
    "text": "I love programming in Python. It is a versatile language that is great for data science and web development.",
})

In [16]:
result.to_messages()

[SystemMessage(content='Translate the following into French :', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I love programming in Python. It is a versatile language that is great for data science and web development.', additional_kwargs={}, response_metadata={})]

In [17]:
chain = prompt | model | parser
respond = chain.invoke({
    "language": "French",
    "text": "I love programming in Python. It is a versatile language that is great for data science and web development.",
})

In [18]:
respond

"<think>\n\n</think>\n\nJ'adore programmer en Python. C'est un langage polyvalent, idéal pour la science des données et le développement web."